# Homework 3: Walt Disney DataFrame

### Name: Shayari Peiris
### Collaborator: Vishwesh Srinivasan


DATA 201A 

Fall 2022

Tufts University

## Introduction

**Business Context.** For this case, you are working as a data analytics consultant for The Walt Disney Studios, one of the "Big Five" major film studios located in Burbank, CA. The team that recruited you has manually kept track of metrics and attributes associated with each film produced from 1937 to 2016. You, their hired data consultant, have been tasked with the question: **"Which films have been the most lucrative for our studios and how can we use the success of those films as a signal for driving sales forward?"**

You will answer this question by following the tasks provided at the end of this introdcution. The studio has provided you with access to movie content data for each film, gross income per year, and the voice actors data for each film. First, you will load the datasets and join them into a single data structure. Second, you will conduct an analysis of the dataset by following the five tasks provided below.

**Business Problem.** Your task is to **write code in Python to carry out the requested data analysis**.

**Analytical Context.** You are given the data as three separate CSV files, the preferred data format. You haven't been given much instruction on how to use these data sources, but you may want to merge them for easier analysis. As a consultant, your client has requested the following assignments:

1. Load each dataset and then merge them into a single Pandas DataFrame
2. Identify which films generated the highest total gross income for each decade (1930, 1940, 1950, ...)
3. Determine which films generated the highest total gross income for each genre
4. Taking into account inflation, name the top 10 films that generated the most profit
5. Provide the name of each voice actor that played in each of the top movies from Task 4

*Dataset provide by data.world via user kgarrett* <br/>

## Overview of the data

The data for this case is contained in the Disney datasets folder provided via Canvas. We will be focusing on the tables that belong to the **Characters, Voice Actors, and Total Gross** categories. Below, you'll find a schema for each of these tables. 

Complete documentation for the original data (of which you have only a subset) can be found [here](https://data.world/kgarrett/disney-character-success-00-16/workspace/data-dictionary).

**Disney Characters Table:**
* **movie_title  (string)**: The title of the film once released by The Walt Disney Studios
* **release_date (date)**:   The date the film was released in the format YYYY-MM-DD
* **hero         (string)**: The name of the main protagonist of the film
* **villian      (string)**: The name of the main antagonist of the film
* **song         (string)**: The main theme from the film's released soundtrack


**Disney Voice Actors Table:**
* **character   (string)**: The name of any of the given characters featured in the film
* **voice_actor (string)**: The name of the voice actor playing their associated character
* **movie       (string)**: The title of the film once released by The Walt Disney Studios

**Disney Movies Total Gross Table:**
* **movie_title               (string)**: The title of the film once released by The Walt Disney Studios
* **release_date              (date)**:   The date the film was released in the format YYYY-MM-DD
* **genre                     (string)**: The genre category of the film
* **mpaa_rating               (string)**: The parental rating of the film
* **total_gross               (decimal)**: The amount of income generated by the film by the reported date
* **inflation_adjusted_gross  (decimal)**: The amount of income generated by the film and adjusted for inflation

### Import all necessary libraries

In [373]:
import pandas as pd
import numpy as np
import matplotlib as plt
import math

## 1. Loading the Data

(20 points)

Real-world data is often messy and very dirty. As data practitioners, we often spend our time cleaning and scrubbing data before we even get to the analysis stage. We provide a dataset that is still somewhat dirty but manageable for this assignment. You will be responsible for loading each data source into a single pandas df, though a difficult part of the code has already been written.

**Since our notebook and our datasets folder are located in the same location, we can simply enter the path to read in the dataset. If your notebook and datasets folder are not located in the same folder (also known as repository), then make sure to move them to the same location before running the cell below.**

Complete the cell below to read in each dataset:

In [374]:
movie_characters = pd.read_csv('disney-characters.csv')
voice_actors = pd.read_csv('disney-voice-actors.csv')
total_gross = pd.read_csv('disney_movies_total_gross.csv')

In [375]:
# Run this line before merging your datasets to keep the name of each film clean
movie_characters['movie_title'] = movie_characters['movie_title'].str.replace('\n', '')

Output the first 15 rows of each dataset using 'df.head()', showing each output in a different cell.

In [376]:
#Problem 1: outputting first 15 rows for movie_characters
movie_characters.head(15)

,movie_title,release_date,hero,villian,song
0,Snow White and the Seven Dwarfs,"December 21, 1937",Snow White,Evil Queen,Some Day My Prince Will Come
1,Pinocchio,"February 7, 1940",Pinocchio,Stromboli,When You Wish upon a Star
2,Fantasia,"November 13, 1940",NaN,Chernabog,NaN
3,Dumbo,"October 23, 1941",Dumbo,Ringmaster,Baby Mine
4,Bambi,"August 13, 1942",Bambi,Hunter,Love Is a Song
5,Saludos Amigos,"February 6, 1943",Donald Duck,NaN,Saludos Amigos\n
6,The Three Caballeros,"February 3, 1945",Donald Duck,NaN,NaN
7,Make Mine Music,"April 20, 1946",NaN,NaN,NaN
8,Fun and Fancy Free,"September 27, 1947",Mickey Mouse,Willie the Giant,NaN
9,Melody Time,"May 27, 1948",NaN,NaN,Little Toot


In [377]:
#Problem 1 continued: outputting first 15 rows for voice_actors

voice_actors.head(15)

,character,voice-actor,movie
0,Abby Mallard,Joan Cusack,Chicken Little
1,Abigail Gabble,Monica Evans,The Aristocats
2,Abis Mal,Jason Alexander,The Return of Jafar
3,Abu,Frank Welker,Aladdin
4,Achilles,None,The Hunchback of Notre Dame
5,Adella,Sherry Lynn,The Little Mermaid
6,Adorabeezle Winterpop,None,Wreck-It Ralph
7,The Agent,Greg Germann,Bolt
8,Agent Wendy Pleakley,Kevin McDonald,Lilo & Stitch
9,Ajax the Gorilla,None,Donald Duck and the Gorilla


In [378]:
#Problem 1 continued: outputting first 15 rows for total_gross

total_gross.head(15)

,movie_title,release_date,genre,MPAA_rating,total_gross,inflation_adjusted_gross
0,Snow White and the Seven Dwarfs,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251"
1,Pinocchio,"Feb 9, 1940",Adventure,G,"$84,300,000","$2,188,229,052"
2,Fantasia,"Nov 13, 1940",Musical,G,"$83,320,000","$2,187,090,808"
3,Song of the South,"Nov 12, 1946",Adventure,G,"$65,000,000","$1,078,510,579"
4,Cinderella,"Feb 15, 1950",Drama,G,"$85,000,000","$920,608,730"
5,"20,000 Leagues Under the Sea","Dec 23, 1954",Adventure,NaN,"$28,200,000","$528,279,994"
6,Lady and the Tramp,"Jun 22, 1955",Drama,G,"$93,600,000","$1,236,035,515"
7,Sleeping Beauty,"Jan 29, 1959",Drama,NaN,"$9,464,608","$21,505,832"
8,101 Dalmatians,"Jan 25, 1961",Comedy,G,"$153,000,000","$1,362,870,985"
9,The Absent Minded Professor,"Mar 16, 1961",Comedy,NaN,"$25,381,407","$310,094,574"


The cell below uses the pandas 'merge' function to merge these individual DataFrames into a single DataFrame.

In [379]:
# DO NOT EDIT
# The following code merges the datasets into a single pandas df
left_merge = pd.merge(movie_characters, total_gross, on='movie_title')
# We rename the 'movie_title' column to 'movies' to join the datasets
left_merge = left_merge.rename(columns={"movie_title": "movie"})
disney_data = pd.merge(left_merge, voice_actors, on='movie')
# Generate a unique film ID with random seed for each film
# This seeding approach has been improved in 2021
np.random.seed(2021) 
ID_range = np.arange(100000, 900000)
IDs = {m: np.random.choice(ID_range, 1, replace=False)[0] for m in set(disney_data.movie)}
# Assign the unique ID to each film
disney_data['film_id'] = ''
for film, ID in IDs.items():
    disney_data.loc[disney_data['movie'] == film,'film_id'] = str(ID)
# We observe multiple rows of same film with different characters below
disney_data.head()

,movie,release_date_x,hero,villian,song,release_date_y,genre,MPAA_rating,total_gross,inflation_adjusted_gross,character,voice-actor,film_id
0,Snow White and the Seven Dwarfs,"December 21, 1937",Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251",Bashful,Scotty Mattraw,482267
1,Snow White and the Seven Dwarfs,"December 21, 1937",Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251",Doc,Roy Atwell,482267
2,Snow White and the Seven Dwarfs,"December 21, 1937",Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251",Dopey,Eddie Collins,482267
3,Snow White and the Seven Dwarfs,"December 21, 1937",Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251",Grumpy,Pinto Colvig,482267
4,Snow White and the Seven Dwarfs,"December 21, 1937",Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,"$184,925,485","$5,228,953,251",Happy,Otis Harlan,482267


You can run the cell below to see the unique IDs we generated for each film:

In [380]:
# DO NOT EDIT
IDs

{'The Rescuers': 219262,
 'Frozen': 550486,
 'Beauty and the Beast': 130897,
 'Big Hero 6': 329503,
 'Brother Bear': 693653,
 'Sleeping Beauty': 836088,
 'Treasure Planet': 410813,
 'The Hunchback of Notre Dame': 209015,
 'Wreck-It Ralph': 779135,
 'Atlantis: The Lost Empire': 806340,
 'Mulan': 796593,
 'The Princess and the Frog': 407506,
 'Cinderella': 736543,
 'Meet the Robinsons': 694537,
 'Moana': 339918,
 'The Sword in the Stone': 364740,
 'The Lion King': 882484,
 'Lady and the Tramp': 821764,
 'Oliver & Company': 703293,
 'The Fox and the Hound': 683327,
 'The Black Cauldron': 630281,
 'The Rescuers Down Under': 892987,
 'Zootopia': 251627,
 'Alice in Wonderland': 787062,
 'Lilo & Stitch': 645858,
 'Snow White and the Seven Dwarfs': 482267,
 'Tarzan': 464805,
 'The Jungle Book': 576980,
 'Home on the Range': 149306,
 'The Great Mouse Detective': 320542,
 'Hercules': 230805,
 "The Emperor's New Groove": 701647,
 'Fantasia': 149036,
 'The Little Mermaid': 120461,
 'Aladdin': 3357

Run the following cell to perform some string data cleaning on our Disney dataset:

In [381]:
# DO NOT EDIT

# Remove currency symbol from all values in the columns we want as numeric
disney_data["total_gross"] = disney_data["total_gross"].str[1:]
disney_data["inflation_adjusted_gross"] = disney_data["inflation_adjusted_gross"].str[1:]

# Remove all commas from strings
disney_data['total_gross'] = disney_data['total_gross'].str.replace(',', '')
disney_data['inflation_adjusted_gross'] = disney_data['inflation_adjusted_gross'].str.replace(',', '')

# Convert to correct dtype to avoid future bugs
disney_data["total_gross"] = pd.to_numeric(disney_data["total_gross"])
disney_data["inflation_adjusted_gross"] = pd.to_numeric(disney_data["inflation_adjusted_gross"])

In [382]:
# DO NOT EDIT

disney_data.dtypes

movie                       object
release_date_x              object
hero                        object
villian                     object
song                        object
release_date_y              object
genre                       object
MPAA_rating                 object
total_gross                  int64
inflation_adjusted_gross     int64
character                   object
voice-actor                 object
film_id                     object
dtype: object

## 2. Film with highest total gross income for each decade (1930, 1940, 1950, ...)

(20 points)

To analyze a dataset using dates, we need to convert the dtype of our column 'release_date_x' to the 'Datetime' format in pandas.
We can use ['to_datetime'](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) to accomplish this goal by converting our column 'release_date_x' to datetime. It is currently type 'object' which is an ambiguous dtype used for strings or mixed values in pandas.

Run the cell below to see the current dtype of the column.

In [383]:
disney_data.release_date_x

0      December 21, 1937
1      December 21, 1937
2      December 21, 1937
3      December 21, 1937
4      December 21, 1937
             ...        
595    November 23, 2016
596    November 23, 2016
597    November 23, 2016
598    November 23, 2016
599    November 23, 2016
Name: release_date_x, Length: 600, dtype: object

Run the following cell to convert our release dates to the Datetime dtype:

In [384]:
disney_data['release_date_x'] = pd.to_datetime(disney_data['release_date_x'])

Now, the dtype of our column 'release_date_x' should say 'datetime64[ns]' as seen below. You can try the 'to_datetime' function with different formats by reading the pandas documentation linked to in the previous cell. We will use this column to group our dataset by release date.

In [385]:
disney_data.release_date_x

0     1937-12-21
1     1937-12-21
2     1937-12-21
3     1937-12-21
4     1937-12-21
         ...    
595   2016-11-23
596   2016-11-23
597   2016-11-23
598   2016-11-23
599   2016-11-23
Name: release_date_x, Length: 600, dtype: datetime64[ns]

Now, we can use the 'groupby' function to group our dataset by decade and find the film with the highest gross income for each decade.

1. Group films by 'film_id' and use '.first()' to ensure we are only looking at unique films. 
2. Sort values by year and, most importantly, total gross (non-adjusted).
3. Determine the highest grossing film for each decade available.

In [386]:
# Step 1
# TO DO:
# Group the films by ID using the first appearance
# in the grouping. Store this value in a variable
# called 'by_id'. You will run into errors later
# in the code if you do not use this variable
# name EXACTLY as written.

# Step 2
# TO DO:
# Sort the values by year and then sort by 
# total gross (non-adjusted) value. Store
# this value in a variable called 'by_release_date'.
# Again, follow the exact variable name provided.

# STEP 3
# TO DO:
# To find the highest grossing film, you can do so manually by looking at the values 
# for each film for each decade and record the answers below or use pandas to accomplish 
# this task using Datetime. For the second (and preferred) option, you may need to look up 
# how to set a pandas index with datetime values.

In [387]:
#Problem 2
#Step 1: grouping films by id using the first appearance and storing in by_id

by_id = disney_data.groupby(['film_id']).first()
by_id

,movie,release_date_x,hero,villian,song,release_date_y,genre,MPAA_rating,total_gross,inflation_adjusted_gross,character,voice-actor
film_id,,,,,,,,,,,,
120461,The Little Mermaid,1989-11-17,Ariel,Ursula,Under the Sea,"Nov 15, 1989",Adventure,G,111543479,223726012,Adella,Sherry Lynn
130897,Beauty and the Beast,1991-11-22,Belle,Gaston,Be Our Guest,"Nov 13, 1991",Musical,G,218951625,363017667,Armoire,Jo Anne Worley
149036,Fantasia,1940-11-13,None,Chernabog,None,"Nov 13, 1940",Musical,G,83320000,2187090808,Ben Ali Gator,Jim Cummings
149306,Home on the Range,2004-04-02,Maggie,Alameda Slim,None,"Apr 2, 2004",Comedy,PG,50026353,67910166,Alameda Slim,Randy Quaid
202056,Chicken Little,2005-11-04,Ace Cluck,Foxy Loxy,None,"Nov 4, 2005",Adventure,G,135386665,177954661,Abby Mallard,Joan Cusack
209015,The Hunchback of Notre Dame,1996-06-21,Quasimodo,Claude Frollo,God Help the Outcasts,"Jun 21, 1996",Adventure,G,100138851,190988799,Achilles,None
219262,The Rescuers,1977-06-22,Bernard and Miss Bianca,Madame Medusa,The Journey,"Jun 22, 1977",Adventure,None,48775599,159743914,Bernard,Bob Newheart
221817,The Aristocats,1970-12-24,Thomas and Duchess,Edgar Balthazar,Ev'rybody Wants to Be a Cat,"Apr 24, 1970",Musical,G,55675257,255161499,Abigail Gabble,Monica Evans
230805,Hercules,1997-06-27,Hercules,Hades,Go the Distance,"Jun 13, 1997",Adventure,G,99112101,182029412,Alcmene,Barbara Barrie


In [388]:
#Problem 2 continued
#Step 2: grouping films by release date and total_gross and storing in by_release_date

by_release_date = by_id.sort_values(by=['release_date_x', 'total_gross'])
by_release_date

,movie,release_date_x,hero,villian,song,release_date_y,genre,MPAA_rating,total_gross,inflation_adjusted_gross,character,voice-actor
film_id,,,,,,,,,,,,
482267,Snow White and the Seven Dwarfs,1937-12-21,Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,184925485,5228953251,Bashful,Scotty Mattraw
484637,Pinocchio,1940-02-07,Pinocchio,Stromboli,When You Wish upon a Star,"Feb 9, 1940",Adventure,G,84300000,2188229052,Blue Fairy,Evelyn Venable
149036,Fantasia,1940-11-13,None,Chernabog,None,"Nov 13, 1940",Musical,G,83320000,2187090808,Ben Ali Gator,Jim Cummings
736543,Cinderella,1950-02-15,Cinderella,Lady Tremaine,Bibbidi-Bobbidi-Boo,"Feb 15, 1950",Drama,G,85000000,920608730,Anastasia Tremaine,Lucille Bliss
787062,Alice in Wonderland,1951-07-28,Alice,Queen of Hearts,The Unbirthday Song,"Mar 5, 2010",Adventure,PG,334191110,357063499,Alice,Kathryn Beaumont
821764,Lady and the Tramp,1955-06-22,Lady and Tramp,Si and Am,Bella Notte,"Jun 22, 1955",Drama,G,93600000,1236035515,Al the Alligator,Thurl Ravenscroft
836088,Sleeping Beauty,1959-01-29,Aurora,Maleficent,Once Upon a Dream,"Jan 29, 1959",Drama,None,9464608,21505832,Aurora,Mary Costa
364740,The Sword in the Stone,1963-12-25,Arthur,Madam Mim,Higitus Figitus\n,"Dec 25, 1963",Adventure,None,22182353,153870834,Archimedes,Junius Matthews
576980,The Jungle Book,1967-10-18,Mowgli,Kaa and Shere Khan,The Bare Necessities\n,"Oct 18, 1967",Musical,Not Rated,141843000,789612346,Akela,John Abbott


In [389]:
#Problem 2 continued
#Step 3: creating by_release_date_index by setting index as 'release_date_x'; creating decade_highest_gross and 
#converting index to decades and then taxing the max total gross for each decade

by_release_date_index = by_release_date.set_index('release_date_x')
decade_highest_gross = by_release_date_index.groupby((by_release_date_index.index.year//10)*10)['total_gross'].max()
decade_highest_gross

release_date_x
1930    184925485
1940     84300000
1950    334191110
1960    141843000
1970     55675257
1980    111543479
1990    422780140
2000    145771527
2010    400738009
Name: total_gross, dtype: int64

In [410]:
by_release_date_index

,movie,hero,villian,song,release_date_y,genre,MPAA_rating,total_gross,inflation_adjusted_gross,character,voice-actor
release_date_x,,,,,,,,,,,
1937-12-21,Snow White and the Seven Dwarfs,Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,184925485,5228953251,Bashful,Scotty Mattraw
1940-02-07,Pinocchio,Pinocchio,Stromboli,When You Wish upon a Star,"Feb 9, 1940",Adventure,G,84300000,2188229052,Blue Fairy,Evelyn Venable
1940-11-13,Fantasia,None,Chernabog,None,"Nov 13, 1940",Musical,G,83320000,2187090808,Ben Ali Gator,Jim Cummings
1950-02-15,Cinderella,Cinderella,Lady Tremaine,Bibbidi-Bobbidi-Boo,"Feb 15, 1950",Drama,G,85000000,920608730,Anastasia Tremaine,Lucille Bliss
1951-07-28,Alice in Wonderland,Alice,Queen of Hearts,The Unbirthday Song,"Mar 5, 2010",Adventure,PG,334191110,357063499,Alice,Kathryn Beaumont
1955-06-22,Lady and the Tramp,Lady and Tramp,Si and Am,Bella Notte,"Jun 22, 1955",Drama,G,93600000,1236035515,Al the Alligator,Thurl Ravenscroft
1959-01-29,Sleeping Beauty,Aurora,Maleficent,Once Upon a Dream,"Jan 29, 1959",Drama,None,9464608,21505832,Aurora,Mary Costa
1963-12-25,The Sword in the Stone,Arthur,Madam Mim,Higitus Figitus\n,"Dec 25, 1963",Adventure,None,22182353,153870834,Archimedes,Junius Matthews
1967-10-18,The Jungle Book,Mowgli,Kaa and Shere Khan,The Bare Necessities\n,"Oct 18, 1967",Musical,Not Rated,141843000,789612346,Akela,John Abbott


In [390]:
#Problem 2 continued
#Step 3: merging decade_highest_gross and by_release_date to match film names and highest total gross for each decade

by_decade = pd.merge(decade_highest_gross, by_release_date, on = 'total_gross')
by_decade

,total_gross,movie,release_date_x,hero,villian,song,release_date_y,genre,MPAA_rating,inflation_adjusted_gross,character,voice-actor
0,184925485,Snow White and the Seven Dwarfs,1937-12-21,Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,5228953251,Bashful,Scotty Mattraw
1,84300000,Pinocchio,1940-02-07,Pinocchio,Stromboli,When You Wish upon a Star,"Feb 9, 1940",Adventure,G,2188229052,Blue Fairy,Evelyn Venable
2,334191110,Alice in Wonderland,1951-07-28,Alice,Queen of Hearts,The Unbirthday Song,"Mar 5, 2010",Adventure,PG,357063499,Alice,Kathryn Beaumont
3,141843000,The Jungle Book,1967-10-18,Mowgli,Kaa and Shere Khan,The Bare Necessities\n,"Oct 18, 1967",Musical,Not Rated,789612346,Akela,John Abbott
4,55675257,The Aristocats,1970-12-24,Thomas and Duchess,Edgar Balthazar,Ev'rybody Wants to Be a Cat,"Apr 24, 1970",Musical,G,255161499,Abigail Gabble,Monica Evans
5,111543479,The Little Mermaid,1989-11-17,Ariel,Ursula,Under the Sea,"Nov 15, 1989",Adventure,G,223726012,Adella,Sherry Lynn
6,422780140,The Lion King,1994-06-24,Simba,Scar,Circle of Life,"Jun 15, 1994",Adventure,G,761640898,Banzai,Cheech Marin
7,145771527,Lilo & Stitch,2002-06-21,Lilo and Stitch,None,He Mele No Lilo\n,"Jun 21, 2002",Adventure,PG,211506702,Agent Wendy Pleakley,Kevin McDonald
8,400738009,Frozen,2013-11-27,Elsa,Prince Hans,Let It Go\n,"Nov 22, 2013",Adventure,PG,414997174,Anna,Kristen Bell


Provide the name of the highest-grossing film for each decade from the 1930's to the 2010's.

**_____YOUR ANSWER HERE_____:**

1930s: Snow White and the Seven Dwarfs
<HR/>
1940s: Pinocchio
<HR/>
1950s: Alice and Wonderland
<HR/>
1960s: The Jungle Book
<HR/>
1970s: The Aristocats
<HR/>
1980s: The Little Mermaid
<HR/>
1990s: The Lion King
<HR/>
2000s: Lilo and Stitch
<HR/>
2010s: Frozen

Make sure that the value for the cell below is equal to 1. This means that each film is only counted once in the above cell. If so, then we are successfully analyzing only a single movie and not using duplicates in our analysis. We can then move forward with our values for this part of the analysis, assuming we grouped and sorted correctly.

In [391]:
max(by_release_date.index.value_counts())

1

## 3. Films with highest total gross income for each genre

(20 points)

Our next task is to find out which film generated the highest total gross for each genre. In this dataset, we have the four following genres:
<ol>
    <li> Adventure </li>
    <li> Musical </li>
    <li> Comedy </li>
    <li> Drama </li>
</ol>

Use the combination of 'pd.sort_values()' and 'pd.groupby()' as completed in the previous task to find these four  movies. Make sure you are only using functions available in pandas and/or numpy for your code. You should only have one single movie for each genre (i.e. a total of 4 movies). In the unlikely case of a tie, choose the older movie.

**WARNING: Make sure you are using the 'disney_data' df for this section. The movie Cinderella had a 2015 reboot but does not have a different name than the original or a different ID. As such, you will need to use the entire Disney film dataset to find the highest grossing movies by genre.**

In [392]:
by_id.genre.value_counts()

Adventure    30
Musical       5
Comedy        4
Drama         3
Name: genre, dtype: int64

In [393]:
#Problem 3
#sorting and grouping values in disney_data to get a df with highest total gross for each genre

total_gross_all = disney_data.sort_values(['total_gross']).groupby(['genre'])['total_gross'].max()
total_gross_all

genre
Adventure    422780140
Comedy       217350219
Drama        201151353
Musical      218951625
Name: total_gross, dtype: int64

In [394]:
#Problem 3 continued
#merging total_gross_all and disney_data to match movie names to highest total gross for each genre

by_genre = pd.merge(total_gross_all, disney_data, on = 'total_gross').groupby('genre').first()
by_genre

,total_gross,movie,release_date_x,hero,villian,song,release_date_y,MPAA_rating,inflation_adjusted_gross,character,voice-actor,film_id
genre,,,,,,,,,,,,
Adventure,422780140,The Lion King,1994-06-24,Simba,Scar,Circle of Life,"Jun 15, 1994",G,761640898,Banzai,Cheech Marin,882484
Comedy,217350219,Aladdin,1992-11-25,Aladdin,Jafar,A Whole New World,"Nov 11, 1992",G,441969178,Abu,Frank Welker,335712
Drama,201151353,Cinderella,1950-02-15,Cinderella,Lady Tremaine,Bibbidi-Bobbidi-Boo,"Mar 13, 2015",PG,201151353,Anastasia Tremaine,Lucille Bliss,736543
Musical,218951625,Beauty and the Beast,1991-11-22,Belle,Gaston,Be Our Guest,"Nov 13, 1991",G,363017667,Armoire,Jo Anne Worley,130897


Provide the names of the four films that generated the highest total gross income for each genre.

**_____YOUR ANSWER HERE_____:** 

Adventure: The Lion King
<HR/>
Comedy: Aladdin
<HR/>
Drama: Cinderella
<HR/>
Musical: Beauty and the Beast

## 4. 10 Ten Films with Adjustements for Inflation

(20 points)

Similar to the previous section, we want to use our inflation adjusted column to find the highest grossing movies to date. You will need to sort your values by the inflation adjusted gross column and then group by movie title, selecting the first movie (i.e. .first( ) ) from your groupby object. This process will allow us to avoid any duplicates or double-counting of films. From there, you can sort by the gross one last time and use .head() to find the 10 top films. Remember, we are still using the full dataset for these remaining tasks.

* Sort values by inflation adjusted gross
* Group this sorted df by movie title to avoid double-counting
* Use .first() to avoid double-counting films in your groupby
* Sort your data by inflation adjusted gross and choose the top 10 films

In [395]:
#Problem 4

# Complete the code to for the variable
# 'top_ten_adjusted' which maintains
# the values for the top 10 films by
# highest gross adjusted for inflation.

# The variable 'sorted_films' contains
# a view of the dataset where the rows
# only contain one appearance of each film,
# eliminating duplicate appearances of films.
# However, you need to find the top 10 films 
# from this pruned list.

sorted_films = disney_data.sort_values('inflation_adjusted_gross', ascending=False).groupby('movie').first()
top_ten_adjusted = sorted_films.sort_values('inflation_adjusted_gross', ascending=False).head(10)
top_ten_adjusted

,release_date_x,hero,villian,song,release_date_y,genre,MPAA_rating,total_gross,inflation_adjusted_gross,character,voice-actor,film_id
movie,,,,,,,,,,,,
Snow White and the Seven Dwarfs,1937-12-21,Snow White,Evil Queen,Some Day My Prince Will Come,"Dec 21, 1937",Musical,G,184925485,5228953251,Bashful,Scotty Mattraw,482267
Pinocchio,1940-02-07,Pinocchio,Stromboli,When You Wish upon a Star,"Feb 9, 1940",Adventure,G,84300000,2188229052,J. Worthington Foulfellow,Walter Catlett,484637
Fantasia,1940-11-13,None,Chernabog,None,"Nov 13, 1940",Musical,G,83320000,2187090808,Chernabog,Corey Burton,149036
Lady and the Tramp,1955-06-22,Lady and Tramp,Si and Am,Bella Notte,"Jun 22, 1955",Drama,G,93600000,1236035515,Beaver,Stan Freberg,821764
Cinderella,1950-02-15,Cinderella,Lady Tremaine,Bibbidi-Bobbidi-Boo,"Feb 15, 1950",Drama,G,85000000,920608730,Major the Horse,None,736543
The Jungle Book,1967-10-18,Mowgli,Kaa and Shere Khan,The Bare Necessities\n,"Oct 18, 1967",Musical,Not Rated,141843000,789612346,King Louie,Louis Prima,576980
The Lion King,1994-06-24,Simba,Scar,Circle of Life,"Jun 15, 1994",Adventure,G,422780140,761640898,Ed,Jim Cummings,882484
Aladdin,1992-11-25,Aladdin,Jafar,A Whole New World,"Nov 11, 1992",Comedy,G,217350219,441969178,Aladdin,Scott Weinger; Brad Kane,335712
Frozen,2013-11-27,Elsa,Prince Hans,Let It Go\n,"Nov 22, 2013",Adventure,PG,400738009,414997174,Anna,Kristen Bell,550486


In [396]:
#Problem 4 continued: printing list of 10 movies with highest inflation adjusted gross

print(top_ten_adjusted['inflation_adjusted_gross'])

movie
Snow White and the Seven Dwarfs    5228953251
Pinocchio                          2188229052
Fantasia                           2187090808
Lady and the Tramp                 1236035515
Cinderella                          920608730
The Jungle Book                     789612346
The Lion King                       761640898
Aladdin                             441969178
Frozen                              414997174
Beauty and the Beast                363017667
Name: inflation_adjusted_gross, dtype: int64


Provide the names of the 10 highest-grossing films (adjusted for inflation) from The Walt Disney Studios catalog.

**_____YOUR ANSWER HERE_____:**

Snow White and the Seven Dwarfs
<HR/>
Pinocchio
<HR/>
Fantasia
<HR/>
Lady and the Tramp
<HR/>
Cinderella
<HR/>
The Jungle Book
<HR/>
The Lion King
<HR/>
Aladdin
<HR/>
Frozen
<HR/>
Beauty and the Beast

## 5. Voice actors for each of the top movies from Task 4

(20 points)

From the top ten films in the previous section, create a dictionary of pairs where the key is the name of the film and the value is the list of voice actors that starred in that film.

Example format:

* {  film : voice actors}

Part of this code has already been written for you. It will be your task to get the data into a dictionary format.

In [405]:
#Problem 5: creating a for loop that will create a dictionary containing pairs of films and their voice actors
  # TO DO:
    # Complete the code by implementing
    # the rest of this for loop. Your
    # 'voice_actors' should be a list
    # of dictionaries contain the film
    # as a key and a list of voice actors
    # as a value.
    # You set an item in a dictionary by doing dictionary[key] = item
    
voice_actors = []

film_voice_actors = {}

for film in top_ten_adjusted.index:
    voice_actors = (disney_data[disney_data['movie'] == film]['voice-actor'].to_list())
    film_voice_actors[film] = voice_actors

In [406]:
film_voice_actors

{'Snow White and the Seven Dwarfs': ['Scotty Mattraw',
  'Roy Atwell',
  'Eddie Collins',
  'Pinto Colvig',
  'Otis Harlan',
  'Stuart Buchanan',
  'Moroni Olsen',
  'Harry Stockwell',
  'Lucille La Verne',
  'Pinto Colvig',
  'Billy Gilbert',
  'Adriana Caselotti'],
 'Pinocchio': ['Evelyn Venable',
  'None',
  'Charles Judels',
  'Mel Blanc',
  'Christian Rub',
  'Mel Blanc',
  'Walter Catlett',
  'Cliff Edwards; Eddie Carroll',
  'Frankie Darro',
  'Thurl Ravenscroft',
  'Dickie Jones',
  'Charles Judels'],
 'Fantasia': ['Jim Cummings',
  'Corey Burton',
  'Tress MacNeille',
  'Tress MacNeille',
  'None',
  'Corey Burton'],
 'Lady and the Tramp': ['Thurl Ravenscroft',
  'Verna Felton',
  'Stan Freberg',
  'Peggy Lee',
  'Lee Millar',
  'Bill Thompson',
  'Barbara Luddy',
  'Peggy Lee',
  'Peggy Lee',
  'Larry Roberts',
  'Bill Baucom'],
 'Cinderella': ['Lucille Bliss',
  'Jimmy MacDonald; Frank Welker',
  'Ilene Woods',
  'Don Barclay',
  'Rhoda Williams; Russi Taylor',
  'Verna Felt